In [1]:
import time
import os
import subprocess
import sys
import re
import argparse
import collections
import gzip
import math
import shutil
import matplotlib.pyplot as plt
import wandb
import numpy as np
import time
from datetime import datetime
import random
from wandb.keras import WandbCallback


import tensorflow as tf
import tensorflow.experimental.numpy as tnp
import tensorflow_addons as tfa
from tensorflow import strings as tfs
from tensorflow.keras import mixed_precision

## custom modules
import src.genformer1 as genformer
import src.metrics as metrics
import src.optimizers as optimizers
import src.schedulers as schedulers
import src.utils as utils

import training_utils

2022-05-24 16:12:47.158336: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-24 16:12:47.158393: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
#mixed_precision.set_global_policy('mixed_bfloat16')

In [4]:
heads_dict = {}
orgs = ["hg","mm"]
for k, org in enumerate(orgs):
    heads_dict[org] = int(k)

model = genformer.genformer(kernel_transformation="softmax_kernel_transformation",
                            dropout_rate=0.3,
                            final_out_length=20,
                            num_heads=4,
                            numerical_stabilizer=0.0000001,
                            nb_random_features=256,
                            hidden_size=96,
                            d_model=96,
                            dim=24,
                            max_seq_length = 4096,
                            rel_pos_bins=192,
                            widening = 2, ## ratio between first and second dense layer units in transformer block
                            conv_filter_size=15,
                            transformer_depth=3,
                            momentum=0.90,
                            channels_list=[24,24,24,24,48,48,48,48,96,96,96],
                            kernel_regularizer=0.001,
                            heads_dict=heads_dict)

# Restore the weights
model.load_weights("gs://picard-testing-176520/genformer_test_model22/checkpoint")

In [5]:
options = tf.data.Options()
options.experimental_distribute.auto_shard_policy = tf.data.experimental.AutoShardPolicy.OFF
options.deterministic=False
options.experimental_threading.max_intra_op_parallelism = 1

val_dataset_aorta = training_utils.return_dataset_validation("gs://picard-testing-176520/TSS_164k_82kstride_ATAC_scaled_blacklist0.10_4096outputres_logTPM/val/HG_AORTA_THORACIC_ENCDO451RUA_U.tfr",
                                                             "hg",
                                                             1,
                                                             163840,
                                                             40,
                                                             20,
                                                             10,
                                                             options,
                                                             8,
                                                             5)

In [6]:
iter_aorta = iter(val_dataset_aorta)

In [7]:
test_input = next(iter_aorta)

In [10]:
model.predict_on_batch(tf.cast(test_input['inputs'],dtype=tf.float32))

[{'hg': <tf.Tensor: shape=(1, 20, 1), dtype=float32, numpy=
  array([[[0.00566051],
          [0.01476061],
          [0.00858937],
          [0.00292632],
          [0.00498626],
          [0.00300058],
          [0.00507136],
          [0.00056705],
          [0.0011196 ],
          [0.00364503],
          [0.00599788],
          [0.0020708 ],
          [0.00419474],
          [0.00848599],
          [0.00202148],
          [0.00347165],
          [0.00142598],
          [0.00413181],
          [0.01158943],
          [0.00123154]]], dtype=float32)>,
  'mm': <tf.Tensor: shape=(1, 20, 1), dtype=float32, numpy=
  array([[[7.5885671e-04],
          [6.3647136e-02],
          [6.1786082e-04],
          [1.3593354e-04],
          [1.1401018e-03],
          [9.8025333e-04],
          [9.4932172e-04],
          [1.0099299e-04],
          [1.3680171e-04],
          [1.6595253e-04],
          [6.1500253e-04],
          [2.7673988e-04],
          [7.9403486e-04],
          [1.8415251e-03],
   

In [9]:
model.predict_on_batch(tf.cast(test_input['inputs'],dtype=tf.float32))

[{'hg': <tf.Tensor: shape=(1, 20, 1), dtype=float32, numpy=
  array([[[0.00566051],
          [0.01476061],
          [0.00858937],
          [0.00292632],
          [0.00498626],
          [0.00300058],
          [0.00507136],
          [0.00056705],
          [0.0011196 ],
          [0.00364503],
          [0.00599788],
          [0.0020708 ],
          [0.00419474],
          [0.00848599],
          [0.00202148],
          [0.00347165],
          [0.00142598],
          [0.00413181],
          [0.01158943],
          [0.00123154]]], dtype=float32)>,
  'mm': <tf.Tensor: shape=(1, 20, 1), dtype=float32, numpy=
  array([[[7.5885671e-04],
          [6.3647136e-02],
          [6.1786082e-04],
          [1.3593354e-04],
          [1.1401018e-03],
          [9.8025333e-04],
          [9.4932172e-04],
          [1.0099299e-04],
          [1.3680171e-04],
          [1.6595253e-04],
          [6.1500253e-04],
          [2.7673988e-04],
          [7.9403486e-04],
          [1.8415251e-03],
   

In [29]:
model.return_att_weights(test_input['inputs'])

NameError: in user code:

    File "/home/jupyter/dev/genformer/src/genformer1.py", line 183, in return_att_weights  *
        return self.transformer_stack.return_attention_weights(x,training=False)
    File "/home/jupyter/dev/genformer/src/layers/layers.py", line 549, in return_attention_weights  *
        build(inputs.shape)

    NameError: name 'build' is not defined
